# Testing

## Testing Patterns / Principles

### TDD Principles


- Red → Green → Refactor
  - write a failing test, make it pass, then clean up
- Test small units first; keep tests deterministic (seed RNG)
- Isolate side effects (I/O, network) via dependency injection or mocks
- Favor fast tests; run them in CI on every push

### AAA: Arrange-Act-Assert

- Keep each phase visually separated (blank lines help)
- Assert invariants (shape, monotonicity, conservation) not just values

In [ ]:
def normalize(xs):
    total = sum(xs)
    return [x/total for x in xs] if total else xs

# Arrange
xs = [2, 2, 6]
# Act
ys = normalize(xs)
# Assert
assert abs(sum(ys) - 1.0) < 1e-9
assert len(ys) == len(xs)

### Clean Testing Environment

In [ ]:
# Strategies to isolate tests:
# - Use tmp_path / tempfile for safe file writes
# - Mock environment variables
# - Reset global state (random.seed, numpy seed)
# - Close file handles, sockets, or DB connections after each test
print("Always clean up global or I/O state between tests.")

### Test Organization Patterns

In [ ]:
# Structure large test suites for maintainability:
# tests/
#   ├── test_unit/
#   │     ├── test_math.py
#   │     └── test_utils.py
#   ├── test_integration/
#   │     └── test_api_endpoints.py
#   ├── conftest.py   # pytest fixtures available project-wide
#   └── data/         # golden files, mock data
print("Organize tests by type and use fixtures for shared setup.")

## pytest

### Assertions

In [ ]:
def mean(xs):
    assert len(xs) > 0, "mean() requires non-empty list"
    return sum(xs) / len(xs)

print(mean([1,2,3]))
# mean([])  # would raise AssertionError

### Testing Hooks

In [ ]:
def is_pal(s: str) -> bool:
    """
    Return True if s is a palindrome.

    Examples:
    >>> is_pal('racecar')
    True
    >>> is_pal('abc')
    False
    """
    return s == s[::-1]

assert is_pal("madam") is True
assert is_pal("nope") is False
# To run doctests in a script:
# if __name__ == "__main__":
#     import doctest; doctest.testmod()

### Unit Testing

In [ ]:
# Pytest discovers functions named test_* in files/test modules.
# Example tests (showing the style—run with `pytest -q` in a terminal):

def add(a, b): return a + b

def test_add_basic():
    assert add(2, 3) == 5

### Parametrized Tests

In [ ]:
# Test multiple inputs with one function (concept).
#
# import pytest
# @pytest.mark.parametrize("nums,expected", [
#     ([1,2,3], 6),
#     ([0,0,0], 0),
#     ([-1,1], 0),
# ])
# def test_sum(nums, expected):
#     assert sum(nums) == expected
print("Use pytest.mark.parametrize for testing many cases compactly.")

### Hypothesis testing

In [ ]:
# If Hypothesis is installed: `pip install hypothesis`
# from hypothesis import given, strategies as st
# @given(st.lists(st.integers()))
# def test_reverse_reverse(xs):
#     ys = list(reversed(list(reversed(xs))))
#     assert ys == xs
print("Property-based testing checks invariants across many random inputs.")

### Fuzz Testing

In [ ]:
import random

def reverse_twice(xs):
    return list(reversed(list(reversed(xs))))

for _ in range(5):
    seq = [random.randint(-10,10) for _ in range(5)]
    assert reverse_twice(seq) == seq

print("Fuzz tests run random inputs to catch rare bugs.")

### Snapshot / Golden File Testing

In [ ]:
import json, tempfile, os

def serialize(data):
    return json.dumps(data, indent=2, sort_keys=True)

expected_snapshot = '{"a": 1, "b": 2}'
snapshot = serialize({"a": 1, "b": 2})

assert snapshot.strip() == expected_snapshot.strip()
print("Snapshot test passed.")

### Monkeypatching (temporary overrides)

In [ ]:
# Example using contextlib for a manual override.
import contextlib

def get_user():
    import os
    return os.getenv("USER", "unknown")

@contextlib.contextmanager
def mock_env(var, val):
    import os
    old = os.environ.get(var)
    os.environ[var] = val
    try:
        yield
    finally:
        if old is None:
            del os.environ[var]
        else:
            os.environ[var] = old

with mock_env("USER", "maverick"):
    assert get_user() == "maverick"

print("Manual monkeypatch for env vars successful.")

### Integration Tests

In [ ]:
# Simulate end-to-end flow using multiple functions together.
def load_data():
    return [1, 2, 3]

def process_data(xs):
    return [x * 2 for x in xs]

def main_pipeline():
    return sum(process_data(load_data()))

assert main_pipeline() == 12
print("Integration test successful: pipeline verified.")

### Regression Tests

In [ ]:
# Catch re-introduced bugs by locking in a previously failing case.
def divide(a, b):
    if b == 0:
        raise ZeroDivisionError
    return a / b

def test_divide_regression():
    try:
        divide(1, 0)
    except ZeroDivisionError:
        pass
    else:
        raise AssertionError("Expected ZeroDivisionError")

test_divide_regression()
print("Regression test ensures old bug stays fixed.")

### Fixtures

In [ ]:
# Fixtures help share setup/teardown code cleanly.
# (Example commented so notebook doesn’t error.)
#
# import pytest
#
# @pytest.fixture
# def sample_data():
#     return [1, 2, 3]
#
# def test_sum(sample_data):
#     assert sum(sample_data) == 6
#
# Run with:
# pytest -v
print("pytest fixtures allow shared setup/teardown between tests.")

### Coverage & CLI

In [ ]:
# From terminal:
#   pytest -q
#   pytest -q -k "keyword"          # subset by name
#   pytest -q -x                     # stop after first failure
#   pytest --maxfail=1 --disable-warnings -q
#   coverage run -m pytest && coverage html
print("Run pytest/coverage from terminal; see comments for common commands.")

# Debugging

## breakpoint()

In [ ]:
def buggy_sum(xs):
    total = 0
    for x in xs:
        # if x is None: breakpoint()  # uncomment to drop into pdb
        total += x
    return total

print(buggy_sum([1,2,3]))
# buggy_sum([1,None,3])  # uncomment with breakpoint() to inspect

## Logging

In [ ]:
import logging
logging.basicConfig(level=logging.INFO, format="%(levelname)s:%(message)s")

def divide(a, b):
    logging.debug("divide called with a=%s b=%s", a, b)
    if b == 0:
        logging.error("division by zero")
        raise ZeroDivisionError("b must not be zero")
    res = a / b
    logging.info("result=%s", res)
    return res

print(divide(6, 3))
# divide(1, 0)  # would log error and raise

## warnings

In [ ]:
import warnings

def old_api():
    warnings.warn("old_api is deprecated; use new_api", DeprecationWarning, stacklevel=2)
    return 42

# By default, DeprecationWarning may be hidden. Make it visible:
warnings.simplefilter("default", DeprecationWarning)
print(old_api())

## Tracing Exceptions

In [ ]:
def safe_parse_int(s: str, default=None):
    try:
        return int(s)
    except ValueError as e:
        # attach context, keep original traceback
        raise ValueError(f"Cannot parse int from {s!r}") from e

print(safe_parse_int("10"))
# safe_parse_int("ten")  # would raise with helpful message

## Timing

In [ ]:
import timeit

def slow():
    return sum(i*i for i in range(10_000))

print(timeit.timeit(slow, number=100))  # seconds for 100 runs
# In notebooks you can also use:
# %timeit slow()

## Profiling

In [ ]:
import cProfile, pstats, io

def work(n=30_000):
    s = 0
    for i in range(n):
        s += (i % 7) * (i % 11)
    return s

pr = cProfile.Profile()
pr.enable()
_ = work()
pr.disable()

s = io.StringIO()
ps = pstats.Stats(pr, stream=s).sort_stats("cumtime")
ps.print_stats(10)           # top 10 entries
print(s.getvalue().splitlines()[0:15])  # show first few lines

## Determinism

In [ ]:
import random
random.seed(123)
vals = [random.randint(1, 3) for _ in range(5)]
print(vals)  # stable across runs when seeded